## Cell 1: Import Libraries

In [3]:
"""
@author: Albert
"""
import sys
sys.path.append(".")
sys.path.append("..")

import numpy as np
import pandas as pd
import time
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline
from Modules.Models import Models
import warnings
warnings.filterwarnings("ignore")

## Cell 2: Initialising algorithm and required transformations

In [4]:
numberOfClasses = 7
pretrained = True
modelArchitecture = "densenet121"
net = Models(numberOfClasses = 7, pretrained = pretrained, model = modelArchitecture)
modelName = "Model/ABIDLA.tar"
state = torch.load(modelName)
net.load_state_dict(state["model"])
device = torch.device("cpu")
net.to(device)
net.eval()

softmax = nn.Softmax()

transformCompose = transforms.Compose([ transforms.ToPILImage(),
                                        transforms.Scale((224,224)),
                                        transforms.ToTensor()])

## Cell 3
### Please copy images to the Data folder and run the below cell to save ABIDLA's predictions in Outputs folder.

In [5]:
classes = ["Beer Cup", "Beer Bottle", "Beer Can", 
           "Wine", "Champagne", "Undecided", "Other"]

imageDir = "./Data/"
listOfImages = os.listdir(imageDir)
listOfImages.sort(key=lambda x: x[:-4])
totalNumberOfImages = len(listOfImages)

t0 = time.time()

outputFilename = "Outputs/ABIDLAPredictions.csv" #Output filename

if os.path.exists(outputFilename):
    print(f"Predictions file already exists in the \'{outputFilename}\'.")
    print(f"Please delete the existing {outputFilename} to rerun the code.")
    
else:
    with open(outputFilename,"w") as file:
        file.write(f"ImageName,ModelPrediction1,Probability1(%),ModelPrediction2,Probability2(%)")
        print(f"Displaying ABIDLA predictions on first 5 images:")
        
        for e, imageName in enumerate(listOfImages):
            originalImageName = imageName
            imageName = os.path.join(imageDir,imageName)
            image = Image.open(imageName).convert("RGB")
            if e < 5:
                plt.figure()
                imshow(image)
                plt.pause(0.0001)
            image = np.asarray(image)
            image = transformCompose(image)
            image = image[None, :, :,:]
            image = image.to(device)
            output = net(image)

            _, prediction =  torch.max(output,1)
            probabilities = softmax(output)

            top3Classes, indices = probabilities.topk(3)
            highestProbabilty, secondHighestProbability = top3Classes[0,0], top3Classes[0,1]
            thridHighestProbability = top3Classes[0,2]
            BestPrediction, SecondBestPrediction = classes[indices[0,0]], classes[indices[0,1]]
            ThirdBestPrediction = classes[indices[0,2]]
            file.write(f"\n{originalImageName},{BestPrediction},"
                       f"{str(round(float(highestProbabilty), 2)*100)},"
                       f"{SecondBestPrediction}, {str(round(float(secondHighestProbability), 2)*100)}")
            if e < 5:
                print(f"Image name:{originalImageName}")
                print(f"Model predicts : {BestPrediction} " 
                      f"with a probability of {str(round(float(highestProbabilty), 2)*100)} %")
                print(f"Model predicts : {SecondBestPrediction}" 
                      f"with a probability of {str(round(float(secondHighestProbability), 2)*100)} %")
                print(f"Model predicts : {ThirdBestPrediction}" 
                      f"with a probability of {str(round(float(thridHighestProbability), 2)*100)} %")

    t1 = time.time()
    totalTimeTaken = t1 - t0
    print(f"\nModel Prediction is completed")
    print(f"Model Predicted {len(listOfImages)} image(s) " 
          f"in {str(round(totalTimeTaken / 60, 2))} m and {str(round(totalTimeTaken % 60, 2))} seconds.")
    print(f"ABIDLA outputs can be found at {outputFilename}")

Predictions file already exists in the 'Outputs/ABIDLAPredictions.csv'.
Please delete the existing Outputs/ABIDLAPredictions.csv to rerun the code.
